<a href="https://colab.research.google.com/github/akhi9661/IIRS-AOD-session/blob/main/AOD-IIRS-17Dec2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Visualize MODIS (or any GeoTIFF) data

In [ ]:
import rasterio
from rasterio.plot import show
import matplotlib.pyplot as plt

def display_geotiff(path):

    with rasterio.open(geotiff_path) as src:
        data = src.read(1)  # first band
        print("CRS:", src.crs)
        print("Bounds:", src.bounds)
        print("Resolution:", src.res)

        fig, ax = plt.subplots(figsize=(8, 8))
        img = show(data, transform=src.transform, ax=ax, cmap="viridis")

        cbar = plt.colorbar(img.get_images()[0], ax=ax, fraction=0.03, pad=0.02)
        cbar.set_label("AOD (unscaled)")  # label for legend
        plt.show()


# Example
geotiff_path = r"your-file-path.tif"
display_geotiff(geotiff_path)

### Access MAIAC or Sentinel 2 AOT data from Google Earth Engine

##### For MAIAC
- Image Collection: MODIS/061/MCD19A2_GRANULES
- Band: Optical_Depth_055 (for 550nm), Optical_Depth_047 (for 470nm)

##### For Sentinel 2
- Image Collection: COPERNICUS/S2_SR_HARMONIZED
- Band: AOT

In [ ]:
import ee

ee_project = 'your-ee-project'
## More: https://developers.google.com/earth-engine/guides/access

if not ee.data._credentials:
    ee.Authenticate()
if not ee.data._initialized:
    ee.Initialize(project = ee_project)

bbox = ee.Geometry.BBox(73.4, 20.8, 88.5, 32.1)
collection = (ee.ImageCollection("MODIS/061/MCD19A2_GRANULES")
    .filterBounds(bbox)
    .filterDate("2023-01-01", "2023-01-31"))

image = collection.first()
image = image.select("Optical_Depth_055") # Optical depth at 550nm

image_clipped = image.clip(bbox)

# Download locally via URL
print("Generating download URL...")
url = image_clipped.getDownloadURL({"scale": 1000, # MODIS native resolution ~1km
    "crs": "EPSG:4326", "region": bbox})

print("Download link: ", url)

#### Visualize Google Earth Engine data (Map)

In [ ]:
import ee
import folium
import geemap

ee_project = 'your-ee-project'
## More: https://developers.google.com/earth-engine/guides/access

if not ee.data._credentials:
    ee.Authenticate()
if not ee.data._initialized:
    ee.Initialize(project = ee_project)

bbox = ee.Geometry.BBox(73.4, 20.8, 88.5, 32.1)

collection = (ee.ImageCollection("MODIS/061/MCD19A2_GRANULES")
    .filterDate("2023-05-01", "2023-05-15")
    .filterBounds(bbox))

mosaic = collection.mosaic()
aod = mosaic.select("Optical_Depth_055").multiply(0.001)

vis_params = {"min": 0, "max": 1.5, "palette": ["#2c7bb6", "#abd9e9", "#ffffbf", "#fdae61", "#d7191c"],}
m = geemap.Map(center=[26.5, 80], zoom=5)
m.addLayer(aod, vis_params, "MCD19A2_GRANULES AOD Mosaic")

bbox_fc = ee.FeatureCollection([ee.Feature(bbox)])
m.addLayer(bbox_fc, {"color": "black"}, "Bounding Box")

m.addLayerControl()
m